In [1]:
import os
import time
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import mlflow

/Users/jcheung/.virtualenvs/thick-ML-deployment/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## MLFlow
MLFLow has 4 different applications: Tracking, Registry, Models, Projects. In this script, we'll demo all 4 via a local environment. 
1. Tracking: we will log hyper-parameters used and metrics for a transfer-learning model tuned to MNIST.  
2. Registry: after our tuned model is identified, we will register our model. 
3. Models: a stored model must be served to deliver value. Models will be leveraged for inference. 
4. Projects: DS code is packaged to reproduce runs on any platform/machine via Projects.

In [2]:
mlflow.get_tracking_uri()  
# mlflow.set_tracking_uri('http://mlflow-server.kubeflow.svc.cluster.local:5000')  
# mlflow.set_registry_uri('http://minio.kubeflow.svc.cluster.local:9000')  

'file:///Users/jcheung/Documents/GitHub/bookshelf/deployment_demos/mlruns'

### First we'll build all the functions for our ML pipeline.
This'll include functions for data loading, preprocessing, and model training.

In [3]:
def load_tensorflow_dataset(dataset_str: str):
    (xy_train, xy_test), ds_info = tfds.load(
        dataset_str,
        split=['train', 'test'], shuffle_files=True,
        as_supervised=True,
        with_info=True,
    )
    return (xy_train, xy_test)

In [4]:
def preprocess_mnist_tfds(image, label=None):
    # reshape and upsample to 3 channel for transfer learning models
    # ... for when no channel information is present
    if len(image.shape) != 3:
        image = np.dstack((image, image, image))
    # ... for when channel is only 1 dimension
    if image.shape[2] == 1:
        image = tf.image.grayscale_to_rgb(image)
    # normalize pixel values
    image = tf.cast(image, tf.float32) / 255.
    # resize with pad for mobilenetv2
    image = tf.image.resize_with_pad(image, target_height=224, target_width=224)
    return image, label

In [5]:
class MNIST(mlflow.pyfunc.PythonModel):     
    def fit(self, xy_tuple_train, xy_tuple_test, hyperparameters):
        ## Build model
        # class names for mnist hardcoded
        class_names = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    
        # set layer regularization for DNN
        regularizer = tf.keras.regularizers.l1_l2(hyperparameters['l1'], hyperparameters['l2'])

        # load in mobilenetv2 weights and instantiate dense classification head 
        base_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
        layers = [
            hub.KerasLayer(
                base_model,
                input_shape=(224, 224, 3),
                trainable=False,
                name='mobilenet_embedding'),
            tf.keras.layers.Dense(hyperparameters['num_hidden'],
                                  kernel_regularizer=regularizer,
                                  activation='relu',
                                  name='dense_hidden'),
            tf.keras.layers.Dense(len(class_names),
                                  kernel_regularizer=regularizer,
                                  activation='softmax',
                                  name='mnist_prob')
        ]

        self._model = tf.keras.Sequential(layers, name='mnist-classification')

        # compile model 
        self._model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hyperparameters['learning_rate']),
                            loss=tf.keras.losses.SparseCategoricalCrossentropy(
                            from_logits=False),
                            metrics=['accuracy'])
                      
        ## Fit model
        # fit model and save history to model store
        self._train_history = self._model.fit(xy_tuple_train, epochs=hyperparameters['epochs'], validation_data=xy_tuple_test)
        self._model_base = base_model
        
    def predict(self, context, model_input: np.ndarray) -> np.ndarray:
        image, _ = preprocess_mnist_tfds(model_input)
        image = tf.reshape(image, [1, 224, 224, 3])
        return self._model.predict(image).argmax()


                            


### Tracking your model
#### MLFlow has two levels for organizing projects:
1. At the top level we have "experiments". These should be named as "project-task-version" (e.g. mnist-classification)
2. At the lower level we have "runs". A run consists of logging hyperparameters and metrics when models are trained. Multiple runs can be stored within an experiment. 

In [6]:
# mlflow Tracking requires definition of experiment name AND logged params
# Experiment names they should be defined as "project-task-version"

def set_mlflow_experiment(experiment_name:str):
    try:
        experiment_id = mlflow.create_experiment(experiment_name)
    except mlflow.exceptions.MlflowException as e: 
        if str(e) == f"Experiment '{experiment_name}' already exists.":
            print(f'Experiment already exists, setting experiment to {experiment_name}')
            experiment_info = mlflow.set_experiment(experiment_name)
            experiment_id = experiment_info.experiment_id
    experiment = mlflow.get_experiment(experiment_id)
    print("---------------------")
    print('Experiment details are:')
    print("Name: {}".format(experiment.name))
    print("Experiment_id: {}".format(experiment.experiment_id))
    print("Artifact Location: {}".format(experiment.artifact_location))
    print("Creation timestamp: {}".format(experiment.creation_time))
    return experiment_id


experiment_name = "mnist-classification"
experiment_id = set_mlflow_experiment(experiment_name=experiment_name)

Experiment already exists, setting experiment to mnist-classification
---------------------
Experiment details are:
Name: mnist-classification
Experiment_id: 661060572347012195
Artifact Location: file:///Users/jcheung/Documents/GitHub/bookshelf/deployment_demos/mlruns/661060572347012195
Creation timestamp: 1684258817271


In [7]:
# preprocess and define batch sizes for tensorflow 
ds_train, ds_test = load_tensorflow_dataset('mnist')
model = MNIST()
ds_train = ds_train.map(preprocess_mnist_tfds, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.batch(128)
ds_test = ds_test.map(preprocess_mnist_tfds, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128) 


2023-05-22 14:05:05.985743: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# log a base model 
hyperparams = {
    'learning_rate': 0.01,
    'l1': 0.0,
    'l2': 0.0, 
    'num_hidden': 16,
    'epochs': 10}

# Good practice to explicitly define experiment_id and run_name. 
# Experiment_id can be extracted from above. 
# Run name examples (e.g. Linear Regression Default Hyperparams).
mlflow_run_name='MNIST Xfer Learning Base'
with mlflow.start_run(experiment_id=experiment_id, 
                      run_name=mlflow_run_name) as run:
    # You can set autolog for tensorflow model.
    # Note that autolog does not allow logging of any additional params and metrics.
    # We'll choose to do manual logging.
    # mlflow.tensorflow.autolog()

    model.fit(ds_train, ds_test, hyperparams)

    # MLFlow Tracking parameters
    mlflow.log_params(params=hyperparams)
    
    # MLFlow Tracking metrics 
    # Logging metrics for each epoch (housed in dictionary)
    training_history = model._train_history.history
    for epoch in range(0, hyperparams['epochs']):
        insert = {}
        for metric, value in training_history.items():
            insert[metric] = training_history[metric][epoch]
        mlflow.log_metrics(metrics=insert, step=epoch+1)

    # MLFlow tracking artifact (e.g. model file)
    # this will log the model and all its details under run_id/artifacts
    mlflow.pyfunc.log_model(python_model=model,
                           artifact_path="")

    # Close out MLFlow run to prevent any log contamination.
    mlflow.end_run(status='FINISHED')

In [8]:
import optuna
from optuna.integration.mlflow import MLflowCallback


# hyperparameters search using Optuna
# can scale Optuna with Kubernetes https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/004_distributed.html
def objective(trial): 
    """
    Optuna objective function for tuning transfer learning model
    """
    hyperparams = {
        'learning_rate': trial.suggest_float('learning_rate', 0.00001, 0.1, log=True),
        'l1': trial.suggest_float('l1', 0.0, 0.05),
        'l2': trial.suggest_float('l2', 0.0, 0.05),
        'num_hidden': trial.suggest_int('num_hidden', 8, 64),
        'epochs': trial.suggest_int('epochs', 1, 3)
    }

    model.fit(ds_train, ds_test, hyperparams)
    training_history = model._train_history.history
    validation_accuracy = training_history['val_accuracy'][-1]
    return validation_accuracy

POSTGRES_DB="optunadb"
POSTGRES_USER="postgres"
POSTGRES_PASSWORD="mysecretpassword"

optuna_study_name = "mnist-hyperparam-optuna"
optuna_storage_url="postgresql://{}:{}@localhost:5432/{}".format(
            POSTGRES_USER,
            POSTGRES_PASSWORD,
            POSTGRES_DB,
           )
  


In [11]:
try:
    print('loading study...')
    study = optuna.load_study(
        study_name=optuna_study_name,
        storage=optuna_storage_url,
    )
    
except KeyError:
    print('no study found. building from scratch...')
    study = optuna.create_study(
        study_name=optuna_study_name,
        storage=optuna_storage_url,
        pruner=optuna.pruners.HyperbandPruner(),
        direction='maximize')


# create or set an experiment for optuna. Each trial from Optuna is logged as one run in an MLFlow experiment.
experiment_id = set_mlflow_experiment(experiment_name=optuna_study_name)
mlflow_kwargs = {'experiment_id': experiment_id}

# a new experiment name will be created in MLFlow using the Optuna study name
study.optimize(objective,
               n_trials=8,
               n_jobs=2,
              callbacks=[MLflowCallback(metric_name="val_accuracy",
                                        create_experiment=False,
                                        mlflow_kwargs=mlflow_kwargs)]
              )

loading study...
Experiment already exists, setting experiment to mnist-hyperparam-optuna
---------------------
Experiment details are:
Name: mnist-hyperparam-optuna
Experiment_id: 828443912064422063
Artifact Location: file:///Users/jcheung/Documents/GitHub/bookshelf/deployment_demos/mlruns/828443912064422063
Creation timestamp: 1684789437575


/var/folders/7k/0g6mnyq964z2cp592lvzfn480000gr/T/ipykernel_17144/3849037803.py:25: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  callbacks=[MLflowCallback(metric_name="val_accuracy",


Epoch 1/2
Epoch 1/3
469/469 [==============================] - 1124s 2s/step - loss: 16.3877 - accuracy: 0.2699 - val_loss: 14.6970 - val_accuracy: 0.2637
Epoch 2/3


2023-05-22 14:24:59.692933: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


469/469 [==============================] - 1125s 2s/step - loss: 14.9117 - accuracy: 0.2604 - val_loss: 14.6109 - val_accuracy: 0.3203


...

[I 2023-05-22 16:39:34,180] Trial 7 finished with value: 0.2849000096321106 and parameters: {'learning_rate': 0.07387083227571944, 'l1': 0.0035029390922733774, 'l2': 0.0405698172447872, 'num_hidden': 16, 'epochs': 3}. Best is trial 5 with value: 0.8676000237464905.


Epoch 1/3
469/469 [==============================] - 958s 2s/step - loss: 41.8288 - accuracy: 0.2604 - val_loss: 26.9907 - val_accuracy: 0.3723
Epoch 2/3
469/469 [==============================] - 3985s 8s/step - loss: 3.8757 - accuracy: 0.7218 - val_loss: 2.2209 - val_accuracy: 0.7551
Epoch 2/3
469/469 [==============================] - 3977s 8s/step - loss: 17.0162 - accuracy: 0.4443 - val_loss: 9.2888 - val_accuracy: 0.5459
Epoch 3/3
469/469 [==============================] - 1023s 2s/step - loss: 2.1538 - accuracy: 0.7776 - val_loss: 2.0959 - val_accuracy: 0.7753
Epoch 3/3
469/469 [==============================] - 1023s 2s/step - loss: 5.9656 - accuracy: 0.4981 - val_loss: 4.5129 - val_accuracy: 0.4071


[I 2023-05-22 18:03:08,052] Trial 8 finished with value: 0.40709999203681946 and parameters: {'learning_rate': 5.2511164706452605e-05, 'l1': 0.04592869983425784, 'l2': 0.03711551553729608, 'num_hidden': 26, 'epochs': 3}. Best is trial 5 with value: 0.8676000237464905.


469/469 [==============================] - 687s 1s/step - loss: 2.0809 - accuracy: 0.7841 - val_loss: 2.0691 - val_accuracy: 0.7850


[I 2023-05-22 18:14:31,956] Trial 9 finished with value: 0.7850000262260437 and parameters: {'learning_rate': 0.002059020032901681, 'l1': 0.016432402736181007, 'l2': 0.025630669313396637, 'num_hidden': 56, 'epochs': 3}. Best is trial 5 with value: 0.8676000237464905.


In [ ]:
# log the hyper parameter tuned model 
# No need to set experiment ID if you've set above. 
# Set a run name here(e.g. Linear Regression Default Hyperparams).
mlflow_run_name='MNIST Xfer Learning Hyperparam tuned'
with mlflow.start_run(experiment_id=experiment_id, 
                      run_name=mlflow_run_name) as run:
    
    model.train(ds_train, ds_test, hyperparams)

    # MLFlow Tracking parameters
    mlflow.log_params(params=hyperparams)
    
    # MLFlow Tracking metrics 
    # Logging metrics for each epoch (housed in dictionary)
    training_history = model._train_history.history
    for epoch in range(0, hyperparams['epochs']):
        insert = {}
        for metric, value in training_history.items():
            insert[metric] = training_history[metric][epoch]
        mlflow.log_metrics(metrics=insert, step=epoch+1)

    # MLFlow tracking artifact (e.g. model file)
    # mlflow.log_artifact(self._model)

    # we need to define how we use tags for testing or if we even need them...
    mlflow.set_tag(key="test",
                   value="manual-logging")

    mlflow.end_run(status='FINISHED')

### Registering your model
Once you've run a couple experiments (e.g. hyper parameter tuning) we can select the best model and register that. Model registering just means that we decide this our optimized model to save. 

In [ ]:
# search runs 
filtering_clause = 'params.epochs = "10" and params.learning_rate = "0.01"'
run = mlflow.search_runs(
    experiment_names=['mnist-classification'],
    filter_string=filtering_clause,
    max_results=5,
    order_by=["metrics.val_accuracy DESC"],
)

# best performing model run_id. This'll be used to register our model
best_run_id = run['run_id'][0]

In [ ]:
best_run_id

In [ ]:
# register model after a scanning a couple different runs from your experiment 
model_name = f'{experiment.name}'
# mlflow.tensorflow.log_model
mv = mlflow.register_model(model_uri=f"runs:/{best_run_id}/",
                           name=model_name)


### Serving model
After we've registered our model, we can now test inference for that model. 

In [ ]:
# load directly from registry 
# model_version=1
# model_name = f'{experiment.name}'
# model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

# load using artifact path directory
results = mlflow.search_registered_models(filter_string='name = "mnist-classification"')
latest_model_details = results[0].latest_versions[0]
model = mlflow.pyfunc.load_model(model_uri=f'{latest_model_details.source[7:]}')

In [ ]:
# randomly sample and load a MNIST JPEG
import os 
from random import sample
from PIL import Image
fp = '/Users/jcheung/Documents/GitHub/thin-ML-deployment/app/ml/test_images'
files = [f'{fp}/{x}' for x in os.listdir(fp) if x.split('.')[-1] == 'jpg']
filename = sample(files, 1)[0]
image = np.array(Image.open(filename))

# predict using custom mlflow model
predicted = model.predict(image)

# output and compare results
true = filename.split('/')[-1].split('_')[-1][0]

print(f'True:{true} and predicted:{predicted}')

In [ ]:
model.predict(ds_test)

In [ ]:
import kubeflow.katib as katib

# Step 1. Create an objective function.
def objective(hyperparams):
    training_history = model.train(ds_train, ds_test, hyperparams)
    validation_accuracy = training_history.history['val_accuracy'][-1]
    # Katib parses metrics in this format: <metric-name>=<metric-value>.
    print(f"result={result}")

# Step 2. Create HyperParameter search space.
hyperparams = {
    'learning_rate': trial.suggest_float('learning_rate', 0.00001, 0.1, log=True),
    'l1': trial.suggest_float('l1', 0.0, 1),
    'l2': trial.suggest_float('l2', 0.0, 1),
    'num_hidden': katib.search.int(min=8, max=64),
    'epochs': katib.search.int(min=5, max=10)
}

hyperparams = {
    "a": katib.search.int(min=10, max=20),
    "b": katib.search.double(min=0.1, max=0.2)
}

# Step 3. Create Katib Experiment.
katib_client = katib.KatibClient()
name = "tune-experiment"
katib_client.tune(
    name=name,
    objective=objective,
    parameters=parameters,
    objective_metric_name="result",
    max_trial_count=12
)

# Step 4. Get the best HyperParameters.
print(katib_client.get_optimal_hyperparameters(name))